In [ ]:
%pip install 'vanna[chromadb,openai,postgres]'

In [ ]:
from vanna.openai.openai_chat import OpenAI_Chat
from vanna.chromadb.chromadb_vector import ChromaDB_VectorStore
import os
from dotenv import load_dotenv
import json

In [ ]:
load_dotenv()

OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
OPEN_AI_MODEL = os.getenv("OPEN_AI_MODEL")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
DATA_DICTIONARY = os.getenv("DATA_DICTIONARY_PATH")
print(DATA_DICTIONARY)

In [ ]:
data_document = ''
data_dict_path = DATA_DICTIONARY

if data_dict_path:
    with open(data_dict_path) as json_file:
        data_dictionary = json.load(json_file)
        data_document = json.dumps(data_dictionary)


print(data_document)

In [ ]:
class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)

vn = MyVanna(config={'api_key': OPEN_AI_API_KEY, 'model': OPEN_AI_MODEL})

In [ ]:
vn.connect_to_postgres(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, port=DB_PORT)

In [ ]:
data_document

In [ ]:
# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

In [ ]:
vn.train(documentation=data_document)

In [ ]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

In [ ]:
vn.ask(question='What are the top 5 Companies?')